## 1. Create a class and name it Z-test: 

In [1]:
import numpy as np
import scipy.stats as st

class Z_test:
    def __init__(self, sample=[1], pop_mean=1, pop_std=1, alpha=0.05):
        self.sample = np.array(sample)
        self.pop_mean = pop_mean
        self.pop_std = pop_std
        self.alpha = alpha
        
#     def __str__(self):
#         return 'one sample z-test'
        
#     def set_sample(self, sample: iter):
#         self.sample = np.array(sample)
        
#     def set_population(self, pop_mean, pop_std):
#         self.pop_mean = pop_mean
#         self.pop_std = pop_std
    
#     def set_alpha(self, alpha):
#         self.alpha = alpha
        
    def get_statistic(self):
        return (np.mean(self.sample) - self.pop_mean) / (self.pop_std / self.sample.size ** 0.5)
    
    def get_critical_values(self):
        return {'left':st.norm.ppf(self.alpha), 
                'right':st.norm.ppf(1 - self.alpha), 
                'two-tailed':st.norm.ppf(1 - self.alpha / 2)}
    

## 2. Create a class and name it T-test this class includes:
- One-sample: df = n - 1
- Two-sample: df = n1 + n2 - 2
- Paired t-test: df = n - 1
(d bar: is the difference between the two samples
Sd is the standard deviation of the difference between samples)

In [2]:
class T_test:
    def __init__(self, sample1=[1], sample2=[1], pop_mean=1, alpha=0.05):
        self.sample1 = np.array(sample1)
        self.sample2 = np.array(sample2)
        self.pop_mean = pop_mean
        self.alpha = alpha
        
#     def __str__(self):
#         return 't-tests: one-sample, two-samples and paired'
        
#     def set_sample(self, sample: iter):
#         self.sample = np.array(sample)
        
#     def set_population(self, pop_mean):
#         self.pop_mean = pop_mean
    
#     def set_alpha(self, alpha):
#         self.alpha = alpha
        
    def get_statistic_one_sample(self):
        return (np.mean(self.sample1) - self.pop_mean) / (np.std(self.sample1) / self.sample1.size ** 0.5)
    
    def get_statistic_two_samples(self):
        return (np.mean(self.sample1) - np.mean(self.sample2)) / ((np.std(self.sample1, ddof=1) ** 2 / self.sample1.size + np.std(self.sample2, ddof=1) ** 2 / self.sample2.size) ** 0.5)
    
    def get_statistic_paired(self):
        d_bar = self.sample1 - self.sample2
        return np.mean(d_bar) / (np.std(d_bar, ddof=1) / self.sample1.size ** 0.5)
    
    def get_critical_values(self, test_type):
        '''there are 3 test types: "one-sample", "two-samples" and "paired"'''
        if test_type == "two-samples":
            df = self.sample1.size + self.sample2.size - 2
        else:
            df = self.sample1.size - 1
            
        return {'left':st.t.ppf(self.alpha, df), 
                'right':st.t.ppf(1 - self.alpha, df), 
                'two-tailed':st.t.ppf(1 - self.alpha / 2, df)}

## 3. Create a class and name it ANOVA it includes 
- One way 
- Two ways Using the scipy library

In [3]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols


class ANOVA:
    def __init__(self, *args, alpha=0.05):
        self.samples = np.array(args)
        self.alpha = alpha
    
#     def __str__(self):
#         return 'ANOVA test: one-way and two-ways'
        
#     def set_alpha(self, alpha):
#         self.alpha = alpha
    
    def get_one_way_statistic(self):
        k = self.samples.shape[0]
        n_T = self.samples.size
        
        SS_B = sum(np.sum(np.square(row.mean() - self.samples.mean()) * row.size) for row in self.samples)
        df_B = k - 1
        MS_B = SS_B / df_B
        
        SS_W = sum(np.sum(np.square(row - row.mean())) for row in self.samples)
        df_W = n_T - k
        MS_W = SS_W / df_W
        
        return MS_B / MS_W
    
    def get_critical_value(self):
        k = self.samples.shape[0]
        n_T = self.samples.size
        return st.f.ppf(1-self.alpha, k-1, n_T-k)
    
    def get_two_way_statistics(self, keywords):
        '''keys is the dictionary of following format{'var1':['A','B',..], 
            'var2':['A','B',..], 'var_independent':''}'''
        
        keys, values = zip(*keywords.items())
        N = self.samples.size
        df = pd.DataFrame({keys[0]: np.repeat(values[0], int(N / len(values[0]))),
                   keys[1]: np.tile(np.repeat(values[1], int(N / len(values[0]) / len(values[1]))), len(values[0])),
                   keys[2]: self.samples.flatten()})
        
        string = '{} ~ C({}) + C({}) + C({}):C({})'.format(keys[2], keys[0], keys[1], keys[0], keys[1])
        model = ols(string, data=df).fit()
        return sm.stats.anova_lm(model, typ=2)


## 4. Create a class and name it Chi-Square

In [4]:
class Chi_Square:
    def __init__(self, sample, expected_ratio=None, alpha=0.05):
        self.sample = np.array(sample)
        self.alpha = alpha
        if expected_ratio == None:
            self.expected_ratio = np.array([np.sum(self.sample) / self.sample.size])
        else: 
            self.expected_ratio = expected_ratio / np.sum(expected_ratio) * np.sum(self.sample)
    
    def get_statistic(self):
        return sum(np.square(self.sample - self.expected_ratio) / self.expected_ratio)
    
    def get_critical_value(self):
        return st.chi2.ppf(1-self.alpha, df=self.sample.size-1)
            

## 5. Create a class and name it AB testing that inherits the two sample t-test  from the class T-test  

In [5]:
class AB:
    def __init__(self, sample1, sample2):
        self.t_test = T_test(sample1=sample1, sample2=sample2)
    
    def get_statistic(self):
        return self.t_test.get_statistic_two_samples()
    
    def get_critical_values(self):
        return self.t_test.get_critical_values(test_type="two-samples")

# Use these classes to solve the following problems:


## 1. Z-test:
Suppose we are testing a new model of cell-phone and want to compare its bandwidth to the previous model.

Sample = [4.186, 4.439, 4.781, 4.388, 4.947, 4.853, 4.889, 4.682, 4.428,  4.533, 4.557, 4.761, 4.491, 4.334, 4.83 , 4.268, 4.68 , 4.437, 5.382, 5.111, 5.096, 5.232, 5.033, 5.57 , 4.474, 4.789, 4.725, 4.84 , 4.817, 4.438, 4.754, 4.966, 4.285, 4.482, 4.396, 4.418, 4.514, 5.383, 5.264, 4.309, 5.058, 4.392, 4.788, 4.934, 4.967, 4.554, 4.42 , 5. , 5.126, 5.082, 4.944, 4.658]

- State the null and alternative hypotheses.
- Use the Z-test to determine whether the new model has a significantly different bandwidth 4.5 GHz from the previous model with a standard deviation of 0.6 GHz, using a significance level of 0.05.
- Interpret the results


In [6]:
Sample = [4.186, 4.439, 4.781, 4.388, 4.947, 4.853, 4.889, 4.682, 4.428, 4.533, 4.557, 4.761, 4.491, 4.334, 4.83 , 4.268, 4.68 , 4.437, 5.382, 5.111, 5.096, 5.232, 5.033, 5.57 , 4.474, 4.789, 4.725, 4.84 , 4.817, 4.438, 4.754, 4.966, 4.285, 4.482, 4.396, 4.418, 4.514, 5.383, 5.264, 4.309, 5.058, 4.392, 4.788, 4.934, 4.967, 4.554, 4.42 , 5. , 5.126, 5.082, 4.944, 4.658]
H0 = 'New model cell phone bandwith is the same with the old one'
H1 = 'New model cell phone bandwith is different from the old one'

z_test1 = Z_test(sample=Sample, pop_mean=4.5, pop_std=0.6, alpha=0.05)
statistic = z_test1.get_statistic()
critical_2t = z_test1.get_critical_values()['two-tailed']
print('z statistic :', statistic)
print('z critical value two-tailed :', critical_2t)
if statistic < critical_2t:
    print('H0 :', H0)
else:
    print('H1 :', H1)


z statistic : 2.9318216621320863
z critical value two-tailed : 1.959963984540054
H1 : New model cell phone bandwith is different from the old one


## 2. T-test
### A
**One sample T-test:** Suppose we have a new manufacturing process for producing aluminum cans, and we want to test whether the mean weight of the cans produced using this new process is significantly different from the target weight of 15 grams. We randomly sample 30 cans produced using the new process and measure their weights in grams. We obtain the following data: 

Sample = [14.8, 15.2, 15.1, 15.3, 15.0, 14.9, 15.2, 14.8, 15.1, 15.0, 14.9, 14.8, 15.2, 14.9, 15.0, 14.9, 15.1, 15.3, 15.0, 15.1, 14.8, 15.0, 15.2, 15.1, 15.3, 15.1, 15.0, 14.8, 15.2, 15.0]

- State the null and alternative hypotheses.
- Use the appropriate T-test to determine whether the mean weight of the produced cans is equal to the target weight of 15 grams, using a significance level of 0.05.
- Interpret the results


In [7]:
Sample = [14.8, 15.2, 15.1, 15.3, 15.0, 14.9, 15.2, 14.8, 15.1, 15.0, 14.9, 14.8, 15.2, 14.9, 15.0, 14.9, 15.1, 15.3, 15.0, 15.1, 14.8, 15.0, 15.2, 15.1, 15.3, 15.1, 15.0, 14.8, 15.2, 15.0]

H0 = 'Mean weight of the cans produced using this new process is NOT significantly different from 15'
H1 = 'Mean weight of the cans produced using this new process is significantly different from 15'

t_test1 = T_test(sample1=Sample, pop_mean=15, alpha=0.05)

statistic = t_test1.get_statistic_one_sample()
critical_1s = t_test1.get_critical_values('one-sample')['two-tailed']
print('one sample t statistic :', statistic)
print('one sample t critical value two-tailed :', critical_1s)
if -1 * critical_1s < statistic < critical_1s:
    print('H0 :', H0)
else:
    print('H1 :', H1)

one sample t statistic : 1.287746658813943
one sample t critical value two-tailed : 2.0452296421327034
H0 : Mean weight of the cans produced using this new process is NOT significantly different from 15


### B
**Two Sample T-test:** Suppose a food company has developed a new flavor of potato chips and wants to compare it to the current best-selling flavor. The company randomly selects two groups of 20 customers each. The first group is given the new flavor of potato chips, while the second group is given the best-selling flavor. After trying the potato chips, each customer rates the flavor on a scale of 1 to 10. The following are the flavor ratings for the two groups:

New flavor = [8, 7, 9, 6, 7, 8, 9, 7, 8, 7, 6, 8, 7, 9, 8, 7, 6, 9, 8, 7]
Best selling flavor = [6, 7, 8, 6, 7, 6, 7, 6, 8, 7, 6, 7, 6, 8, 7, 6, 7, 8, 6, 7]

- State the null and alternative hypotheses.
- Use the appropriate T-test to determine whether there is a significant difference between the two groups or not, using a significance level of 0.05.
- Interpret the results


In [8]:
New_flavor = [8, 7, 9, 6, 7, 8, 9, 7, 8, 7, 6, 8, 7, 9, 8, 7, 6, 9, 8, 7]
Best_selling_flavor = [6, 7, 8, 6, 7, 6, 7, 6, 8, 7, 6, 7, 6, 8, 7, 6, 7, 8, 6, 7]

H0 = 'A new flavor of potato chips mean rating IS NOT significantly different from current best-seller'
H1 = 'A new flavor of potato chips mean rating IS significantly different from current best-seller'
t_test2 = T_test(sample1=New_flavor, sample2=Best_selling_flavor, alpha=0.05)

statistic = t_test2.get_statistic_two_samples()
critical_2s = t_test2.get_critical_values('two-samples')['two-tailed']
print('two samples t statistic :', statistic)
print('two samples t critical value two-tailed :', critical_2s)
if -1 * critical_2s < statistic < critical_2s:
    print('H0 :', H0)
else:
    print('H1 :', H1)

two samples t statistic : 2.6626213096882494
two samples t critical value two-tailed : 2.024394164575136
H1 : A new flavor of potato chips mean rating IS significantly different from current best-seller


### C
**Paired T-test:** Suppose a company wants to evaluate a new training program for its employees. The company selects 20 employees and measures their productivity before and after the training program. The following are the productivity scores (number of tasks completed per hour) for each employee before and after the training:

Before = [15, 18, 12, 10, 17, 16, 12, 14, 19, 18, 11, 13, 16, 17, 19, 14, 16, 13, 15, 12]
After = [18, 20, 15, 13, 19, 18, 14, 16, 21, 20, 14, 16, 19, 20, 22, 16, 18, 15, 17, 14]

- State the null and alternative hypotheses.
- Use the appropriate T-test to determine whether the new training program has had an effect on employee productivity, with a significance level of 0.05.

In [9]:
Before = [15, 18, 12, 10, 17, 16, 12, 14, 19, 18, 11, 13, 16, 17, 19, 14, 16, 13, 15, 12]
After = [18, 20, 15, 13, 19, 18, 14, 16, 21, 20, 14, 16, 19, 20, 22, 16, 18, 15, 17, 14]

H0 = 'The mean of scores after IS NOT significantly different from the mean of scores before'
H1 = 'The mean of scores after IS significantly different from the mean of scores before'
t_test3 = T_test(sample1=After, sample2=Before, alpha=0.05)

statistic = t_test3.get_statistic_paired()
critical_p = t_test3.get_critical_values('paired')['two-tailed']
print('paired t statistic :', statistic)
print('paired t critical value two-tailed :', critical_p)
if -1 * critical_p < statistic < critical_p:
    print('H0 :', H0)
else:
    print('H1 :', H1)

paired t statistic : 21.354156504062622
paired t critical value two-tailed : 2.093024054408263
H1 : The mean of scores after IS significantly different from the mean of scores before


## 3. ANOVA Test
### A
**One-Way:** Suppose a company has three departments (A, B, and C) and wants to test whether there is a significant difference in salaries between the departments. The company selects 10 employees randomly from each department and records their salaries.

Department A = [55, 60, 50, 58, 63, 62, 57, 56, 61, 59]
Department B = [50, 52, 48, 49, 55, 53, 51, 54, 47, 50]
Department C = [45, 43, 48, 50, 42, 47, 49, 46, 44, 48]

- State the null and alternative hypotheses.
- Use One-Way ANOVA-test to determine whether there is a significant difference in salaries between the three departments.
- Interpret the results



In [10]:
Department_A = [55, 60, 50, 58, 63, 62, 57, 56, 61, 59]
Department_B = [50, 52, 48, 49, 55, 53, 51, 54, 47, 50]
Department_C = [45, 43, 48, 50, 42, 47, 49, 46, 44, 48]

H0 = 'All the department means DO NOT have significant difference'
H1 = 'One of the department means IS significantly different from the others'

anova1 = ANOVA(Department_A, Department_B, Department_C)

statistic = anova1.get_one_way_statistic()
critical_1w = anova1.get_critical_value()
print('ANOVA 1-way statistic :', statistic)
print('ANOVA 1-way critical value :', critical_1w)
if statistic < critical_1w:
    print('H0 :', H0)
else:
    print('H1 :', H1)


ANOVA 1-way statistic : 37.68181818181818
ANOVA 1-way critical value : 3.3541308285291986
H1 : One of the department means IS significantly different from the others


### B
**Two-Way:** Suppose a company has three departments (A, B, and C) and wants to test whether there is a significant difference in salaries between the departments, while also considering the effect of gender. The company selects 10 employees randomly from each department and records their salaries and gender.

Department A:
    Male:   [55k, 60k, 50k, 58k, 63k]
    Female: [62k, 57k, 56k, 61k, 59k]
    
Department B:
    Male:   [50k, 52k, 48k, 49k, 55k]
    Female: [53k, 51k, 54k, 47k, 50k]

Department C:
    Male:   [45k, 43k, 48k, 50k, 42k]
    Female: 47k, 49k, 46k, 44k, 48k]

- State the null and alternative hypotheses.
- Use Two-Way ANOVA-test to determine whether there is a significant difference in salaries between the three departments.
- Interpret the results


In [11]:
data = [55, 60, 50, 58, 63, 62, 57, 56, 61, 59,
50, 52, 48, 49, 55, 53, 51, 54, 47, 50,
45, 43, 48, 50, 42, 47, 49, 46, 44, 48]

anova2 = ANOVA(data)

table = anova2.get_two_way_statistics(keywords={'Department':['A', 'B', 'C'],
                                       'Gender': ['Male', 'Female'],
                                       'Salary':''})
print(table)

                             sum_sq    df          F        PR(>F)
C(Department)            718.466667   2.0  35.104235  7.472321e-08
C(Gender)                  8.533333   1.0   0.833876  3.702368e-01
C(Department):C(Gender)    3.266667   2.0   0.159609  8.533743e-01
Residual                 245.600000  24.0        NaN           NaN


                            sum_sq    df          F        PR(>F)                           
C(Department)            718.466667   2.0  35.104235  7.472321e-08 **IS significantly different**
C(Gender)                  8.533333   1.0   0.833876  3.702368e-01 ISN'T
C(Department):C(Gender)    3.266667   2.0   0.159609  8.533743e-01 ISN'T
Residual                 245.600000  24.0        NaN           NaN

## 4. Chi-Square: Suppose we have a hypothesis that a six-sided die is fair, with each side having an equal probability of landing face up. We roll the die 100 times and record the number of times each side appears: 

Side
Observed Frequency
18 20 16 22 14 10

Given that the expected probabilities are equal to 1/6, Use the class CHi-Square to estimate the goodness-of-fit through chi-square test to determine whether the observed frequencies are consistent with the expected frequencies under the null hypothesis of a fair die.


In [12]:
Observed = [18, 20, 16, 22, 14, 10]

H0 = 'Observed frequencies ARE NOT significantly different from expected'
H1 = 'Observed frequencies ARE significantly different from expected'

chi_squared1 = Chi_Square(sample=Observed)
statistic = chi_squared1.get_statistic()
critical_chi = chi_squared1.get_critical_value()
print('chi-squared statistic :', statistic)
print('chi-squared critical value :', critical_chi)

if statistic < critical_chi:
    print('H0 :', H0)
else:
    print('H1 :', H1)

chi-squared statistic : 5.6
chi-squared critical value : 11.070497693516351
H0 : Observed frequencies ARE NOT significantly different from expected


## 5. A/B testing: Suppose a company has launched a new flavor of soda and wants to test if it has a higher preference than the old flavor. The company conducts a survey with a sample of 30 customers, randomly split into two groups of 15. One group is given the old flavor of soda and the other group is given the new flavor of soda. Each participant rates their preference on a scale of 1 to 10.

Old Flavor = [6, 7, 8, 5, 6, 7, 5, 8, 6, 7, 5, 6, 7, 6, 5]
New Flavor = [8, 9, 7, 8, 9, 6, 7, 8, 7, 8, 7, 8, 9, 6, 8]

- State the null and alternative hypotheses.
- Use two sample t-test to determine whether there is a significant difference in preference between the old and new flavors of soda, with a significance level of 0.05
- Interpret the results


In [13]:
Old_Flavor = [6, 7, 8, 5, 6, 7, 5, 8, 6, 7, 5, 6, 7, 6, 5]
New_Flavor = [8, 9, 7, 8, 9, 6, 7, 8, 7, 8, 7, 8, 9, 6, 8]

H0 = 'A new flavor of soda mean rating IS NOT significantly different from the old'
H1 = 'A new flavor of soda mean rating IS significantly different from the old'

ab1 = AB(sample1=Old_Flavor, sample2=New_Flavor)

statistic = ab1.get_statistic()
critical_ab = ab1.get_critical_values()['two-tailed']
print('two sample two tailed t statistic: ', statistic)
print('two sample two tailed t critical value :', critical_ab)

if -1 * critical_ab < statistic < critical_ab:
    print('H0 :', H0)
else:
    print('H1 :', H1)

two sample two tailed t statistic:  -3.815929867636064
two sample two tailed t critical value : 2.048407141795244
H1 : A new flavor of soda mean rating IS significantly different from the old


## That's all, thanks